NN on dcdc non bdd 

Fully connected


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt
# plot param
%matplotlib inline
plt.rcParams['figure.figsize'] = (22.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

First, manipulate the input and output data

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [2]:
# parse the controller file
f = open('controller.scs', "r")

In [3]:
lines = []

Skip the header part and go straight to the state-action pair

In [4]:
for line in f:
    if '#MATRIX:DATA\n' in line:                
        for line in f: # now you are at the lines you want
            # skip the #BEGIN 
            # read the state-actions
            lines = f.readlines()

In [5]:
# take the state as the train dataset
ltrain_dataset = []
for x in lines:
    ltrain_dataset.append(x.split(' ')[0])
del ltrain_dataset[-1] # delete the string #END at the end of the file 

In [6]:
# convert to the numpy array with float32 data type
train_dataset = np.asarray(ltrain_dataset)
train_dataset = train_dataset.astype(np.float32)
train_dataset.shape

(593089,)

In [7]:
# take the action(s) [column 1:-1] / the rest of the integer except the state
ltrain_label = []
for x in lines:
    ltrain_label.append(x.split(' ')[1:-1])
del ltrain_label[-1] # delete the string #END at the end of the file  

In [8]:
# convert to numpy array, note that the result is still not in one hot encoding format
train_label = np.asarray(ltrain_label)
train_label

array([list(['1']), list(['1']), list(['1']), ..., list(['0']),
       list(['0']), list(['0'])], dtype=object)

Iterate from here

In [9]:
# define number of samples
num_samples = train_dataset.shape[0]
num_samples

593089

In [10]:
# create now array to be filled by the encoded label
train_label_hot = np.zeros([num_samples,2], dtype=np.float32)
train_label_hot.shape

(593089, 2)

In [11]:
ltrain_label[0]

['1']

In [12]:
# encode label to one hot encoding format
for i in range(num_samples):
    if train_label[i] == ['0']:
        train_label_hot[i] = [1, 0]
    elif train_label[i] == ['1']:
        train_label_hot[i] = [0, 1]
    elif train_label[i] == ['0','1']:
        train_label_hot[i] = [1, 1]

Small dataset

In [ ]:
# define the new number of samples for debugging purpose
num_samples = 40
# define batch offset to limit the desired dataset index
batch_off = 20000+16800-87-25
# slice the train label to the desired index
train_label_hot = train_label_hot[batch_off:batch_off + num_samples]
train_label_hot.shape

In [ ]:
# do the same thing to the dataset and add dimension on the column
train_dataset_reform = train_dataset[batch_off:batch_off + num_samples, None]
train_dataset_reform.shape

Execute this to get the full batch

In [13]:
train_dataset_reform = train_dataset[:, None]
train_dataset_reform.shape

(593089, 1)

# NN

In [14]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


In [15]:
train_dataset_reform[0]

array([459.], dtype=float32)

In [16]:
train_dataset_reform[-1]

array([638857.], dtype=float32)

In [17]:
train_label_hot[0]

array([0., 1.], dtype=float32)

In [18]:
train_label_hot[-1]

array([1., 0.], dtype=float32)

In [19]:
num_samples

593089

In [20]:
num_label = 2

In [21]:
batch_size = int(num_samples/100)
hidden_size = 1024
hidden_size2 = 512
hidden_size3 = 256
hidden_size4 = 128

# 20, 100, 500, 1000, 2000

# graph = tf.Graph()
# with tf.device('/device:GPU:0'):
# with graph.as_default():
# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 1))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))
# tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_label))
tf_test_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))

# Variables.
weights1 = tf.Variable(tf.truncated_normal([1, hidden_size]))
biases1 = tf.Variable(tf.zeros([hidden_size]))

# weights2 = tf.Variable(tf.truncated_normal([hidden_size, num_label]))
# biases2 = tf.Variable(tf.zeros([num_label]))

weights2 = tf.Variable(tf.truncated_normal([hidden_size, hidden_size2]))
biases2 = tf.Variable(tf.zeros([hidden_size2]))

weights3 = tf.Variable(tf.truncated_normal([hidden_size2, hidden_size3]))
biases3 = tf.Variable(tf.zeros([hidden_size3]))

# weights4 = tf.Variable(tf.truncated_normal([hidden_size3, num_label]))
# biases4 = tf.Variable(tf.zeros([num_label]))
weights4 = tf.Variable(tf.truncated_normal([hidden_size3, hidden_size4]))
biases4 = tf.Variable(tf.zeros([hidden_size4]))

weights5 = tf.Variable(tf.truncated_normal([hidden_size4, num_label]))
biases5 = tf.Variable(tf.zeros([num_label]))

# Training computation.
logits1 = tf.matmul(tf_train_dataset, weights1) + biases1  
relu_act_func = tf.nn.relu(logits1)
logits2 = tf.matmul(relu_act_func, weights2) + biases2

relu_act_func2 = tf.nn.relu(logits2)
logits3 = tf.matmul(relu_act_func2, weights3) + biases3

relu_act_func3 = tf.nn.relu(logits3)
logits4 = tf.matmul(relu_act_func3, weights4) + biases4

relu_act_func4 = tf.nn.relu(logits4)
logits5 = tf.matmul(relu_act_func4, weights5) + biases5

logits = logits5

# loss calculation
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits5))
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_train_labels, logits=logits5))

# Predictions for the training, validation, and test data.
# train_prediction = tf.nn.softmax(logits5)
train_prediction = tf.sigmoid(logits5)

# loss = tf.reduce_mean(tf.losses.absolute_difference(labels=tf_train_labels, predictions=train_prediction))

loss_summary = tf.summary.scalar('loss', loss)

# Optimizer.
# optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
optimizer = tf.train.AdamOptimizer(2).minimize(loss)

# Predictions for the training, validation, and test data.
# train_prediction = tf.nn.softmax(logits4)

#valid_prediction = tf.nn.softmax(
#    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
# test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(
  #  tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4)+biases4)
 
# test_prediction = tf.sigmoid(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(
#        tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4)+biases4),weights5) + biases5)
     
    # writer = tf.summary.FileWriter('./vehicle', tf.get_default_graph())


In [22]:
batch_size

5930

In [23]:
num_samples

593089

In [24]:
iteration_one_epoch = np.floor(num_samples / batch_size)
iteration_one_epoch

100.0

In [25]:
train_label_hot.shape

(593089, 2)

In [26]:
train_dataset_reform.shape

(593089, 1)

In [27]:
num_steps = int(iteration_one_epoch*10)
loss_history = []

with tf.Session() as session:
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
#with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_reform[offset:(offset + batch_size), :]
        batch_labels = train_label_hot[offset:(offset + batch_size), :]

        # print(offset)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, added_summary, logits_check = session.run(
            [optimizer, loss, train_prediction, loss_summary, logits], feed_dict=feed_dict)

        # writer.add_summary(added_summary)
        loss_history.append(l)
        # print(offset, batch_size)
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print(batch_labels)
        print(predictions)
        # if (step % iteration_one_epoch == 0):
        # print("Minibatch loss at step %d: %f" % (step, l))
            # print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            # print(logits_check)
            # print(weights1.eval())
            # print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    # Do for the test batch
    acc = 0
    for step in range(int(iteration_one_epoch)):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_reform[offset:(offset + batch_size), :]
        batch_labels = train_label_hot[offset:(offset + batch_size), :]

        # print(offset)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        test_predictions = session.run(train_prediction, feed_dict=feed_dict)
        print(offset,offset + batch_size)
        
        print(batch_labels)
        print(test_predictions)
        
        batch_acc = accuracy(test_predictions, batch_labels)
        print("Minibatch accuracy: %.1f%%" % batch_acc)
        acc = acc + batch_acc

    print(acc)
        
        
         

Initialized


InternalError: Blas GEMM launch failed : a.shape=(5930, 1), b.shape=(1, 1024), m=5930, n=1024, k=1
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_Placeholder_0_0/_1, Variable/read)]]
	 [[Node: add_4/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_361_add_4", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'MatMul', defined at:
  File "c:\users\gebruiker\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\gebruiker\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\gebruiker\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_

In [ ]:
533772+59308

In [ ]:
train_label_hot.shape

In [ ]:
predictions

In [ ]:
batch_labels

Small NN

In [ ]:
batch_size = int(num_samples/100)
hidden_size = 4
hidden_size2 = 4
hidden_size3 = 4
hidden_size4 = 4

# 20, 100, 500, 1000, 2000

# graph = tf.Graph()
# with tf.device('/device:GPU:0'):
# with graph.as_default():
# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 1))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))
# tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_label))
tf_test_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))

# Variables.
weights1 = tf.Variable(tf.truncated_normal([1, hidden_size]))
biases1 = tf.Variable(tf.zeros([hidden_size]))

# weights2 = tf.Variable(tf.truncated_normal([hidden_size, num_label]))
# biases2 = tf.Variable(tf.zeros([num_label]))

weights2 = tf.Variable(tf.truncated_normal([hidden_size, hidden_size2]))
biases2 = tf.Variable(tf.zeros([hidden_size2]))

weights3 = tf.Variable(tf.truncated_normal([hidden_size2, hidden_size3]))
biases3 = tf.Variable(tf.zeros([hidden_size3]))

# weights4 = tf.Variable(tf.truncated_normal([hidden_size3, num_label]))
# biases4 = tf.Variable(tf.zeros([num_label]))
weights4 = tf.Variable(tf.truncated_normal([hidden_size3, hidden_size4]))
biases4 = tf.Variable(tf.zeros([hidden_size4]))

weights5 = tf.Variable(tf.truncated_normal([hidden_size4, num_label]))
biases5 = tf.Variable(tf.zeros([num_label]))

# Training computation.
logits1 = tf.matmul(tf_train_dataset, weights1) + biases1  
relu_act_func = tf.nn.relu(logits1)
logits2 = tf.matmul(relu_act_func, weights2) + biases2

relu_act_func2 = tf.nn.relu(logits2)
logits3 = tf.matmul(relu_act_func2, weights3) + biases3

relu_act_func3 = tf.nn.relu(logits3)
logits4 = tf.matmul(relu_act_func3, weights4) + biases4

relu_act_func4 = tf.nn.relu(logits4)
logits5 = tf.matmul(relu_act_func4, weights5) + biases5

logits = logits5

# loss calculation
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits5))
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_train_labels, logits=logits5))

# Predictions for the training, validation, and test data.
# train_prediction = tf.nn.softmax(logits5)
train_prediction = tf.sigmoid(logits5)

# loss = tf.reduce_mean(tf.losses.absolute_difference(labels=tf_train_labels, predictions=train_prediction))

loss_summary = tf.summary.scalar('loss', loss)

# Optimizer.
# optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
optimizer = tf.train.AdamOptimizer(2).minimize(loss)

In [ ]:
num_steps = int(iteration_one_epoch*10)
loss_history = []

with tf.Session() as session:
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
#with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_reform[offset:(offset + batch_size), :]
        batch_labels = train_label_hot[offset:(offset + batch_size), :]

        # print(offset)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, added_summary, logits_check = session.run(
            [optimizer, loss, train_prediction, loss_summary, logits], feed_dict=feed_dict)

        # writer.add_summary(added_summary)
        loss_history.append(l)
        # print(offset, batch_size)
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print(batch_labels)
        print(predictions)
        # if (step % iteration_one_epoch == 0):
        # print("Minibatch loss at step %d: %f" % (step, l))
            # print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            # print(logits_check)
            # print(weights1.eval())
            # print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    # Do for the test batch
    acc = 0
    for step in range(int(iteration_one_epoch)):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_reform[offset:(offset + batch_size), :]
        batch_labels = train_label_hot[offset:(offset + batch_size), :]

        # print(offset)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        test_predictions = session.run(train_prediction, feed_dict=feed_dict)
        print(offset, offset + batch_size)
        batch_acc = accuracy(test_predictions, batch_labels)
        print("Minibatch accuracy: %.1f%%" % batch_acc)
        acc = acc + batch_acc

    print(acc)
        

In [ ]:
#BATCH

In [ ]:
batch_size = int(num_samples)
hidden_size = 1024
hidden_size2 = 512
hidden_size3 = 16

# 20, 100, 500, 1000, 2000

# graph = tf.Graph()

#with graph.as_default():
# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 1))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_label))
# tf_valid_dataset = tf.constant(valid_dataset)
# tf_test_dataset = tf.constant(test_dataset)

# Variables.
weights1 = tf.Variable(tf.truncated_normal([1, hidden_size]))
biases1 = tf.Variable(tf.zeros([hidden_size]))

weights2 = tf.Variable(tf.truncated_normal([hidden_size, hidden_size2]))
# biases2 = tf.Variable(tf.zeros([num_label]))
biases2 = tf.Variable(tf.zeros([hidden_size2]))

weights3 = tf.Variable(tf.truncated_normal([hidden_size2, hidden_size3]))
biases3 = tf.Variable(tf.zeros([hidden_size3]))

weights4 = tf.Variable(tf.truncated_normal([hidden_size3, num_label]))
biases4 = tf.Variable(tf.zeros([num_label]))

# Training computation.
logits1 = tf.matmul(tf_train_dataset, weights1) + biases1  
relu_act_func = tf.nn.relu(logits1)
logits2 = tf.matmul(relu_act_func, weights2) + biases2

relu_act_func2 = tf.nn.relu(logits2)
logits3 = tf.matmul(relu_act_func2, weights3) + biases3

relu_act_func3 = tf.nn.relu(logits3)
logits4 = tf.matmul(relu_act_func3, weights4) + biases4

loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits4))

# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(logits4)

# loss = tf.reduce_mean(tf.losses.absolute_difference(labels=tf_train_labels, predictions=train_prediction))

loss_summary = tf.summary.scalar('loss', loss)

# Optimizer.
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
#optimizer = tf.train.AdamOptimizer(1).minimize(loss)

# Predictions for the training, validation, and test data.
# train_prediction = tf.nn.softmax(logits4)

#valid_prediction = tf.nn.softmax(
#    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
#test_prediction = tf.nn.softmax(
#    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

writer = tf.summary.FileWriter('./vehicle', tf.get_default_graph())

In [ ]:
# BATCH GRADIENT DESCENT
loss_history = []
num_steps = 2

with tf.Session() as session:
#with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_label_hot.shape[0] - batch_size)
    # Generate a minibatch.
    #batch_data = train_dataset_reform[offset:(offset + batch_size), :]
    #batch_labels = train_label_hot[offset:(offset + batch_size), :]
    batch_data = train_dataset_reform
    batch_labels = train_label_hot
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, added_summary = session.run(
        [optimizer, loss, train_prediction, loss_summary], feed_dict=feed_dict)
    
    writer.add_summary(added_summary)
    loss_history.append(l)
    
    # if (step % iteration_one_epoch == 0):
    print("Minibatch loss at step %d: %f" % (step, l))
    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    # print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    # print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))